<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [4]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import pandas as pd

In [5]:
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn

In [7]:
data = pd.read_csv("C:\\Users\\nsuguru\\Desktop\\data.csv")
from sklearn.model_selection import train_test_split

In [20]:
X=data.loc[:,'f1':'f2']
Y=data.loc[:,'label']


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, stratify=Y)

In [10]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(13400, 2)
(6600, 2)
(13400,)
(6600,)


In [37]:
class LossHistory(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        self.history={'loss': [],'acc': [],'val_loss': [],'val_acc': []}
        print('train_begin')
        
    def on_epoch_end(self, epoch, logs={}):
        print('\n','epoch_end')
        ## on end of each epoch, we will get logs and update the self.history dict
        self.history['loss'].append(logs.get('loss'))
        self.history['acc'].append(logs.get('acc'))
        #self.history['auc'].append(logs.get('auc'))
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_acc', -1) != -1:
            self.history['val_acc'].append(logs.get('val_acc'))

In [41]:
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value


In [78]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(2,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=20))(input_layer)
layer2 = Dense(2,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=30))(layer1)
layer3 = Dense(2,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=40))(layer2)
layer4 = Dense(2,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=50))(layer3)
layer5 = Dense(2,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=60))(layer4)

#output layer
output = Dense(1,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=70))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)


#Callbacks
history_own = LossHistory()

optimizer = tf.keras.optimizers.SGD(0.01)

model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

model.fit(X_train,Y_train,epochs=25, validation_data=(X_test,Y_test), batch_size=1000, callbacks=[history_own])


Train on 13400 samples, validate on 6600 samples
train_begin
Epoch 1/25
 1000/13400 [=>............................] - ETA: 1s - loss: 7.5746 - acc: 0.5060
 epoch_end
13400/13400 [==============================] - 0s 23us/sample - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6666 - val_acc: 0.5000
Epoch 2/25
 1000/13400 [=>............................] - ETA: 0s - loss: 7.4980 - acc: 0.5110
 epoch_end
13400/13400 [==============================] - 0s 2us/sample - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6666 - val_acc: 0.5000
Epoch 3/25
 1000/13400 [=>............................] - ETA: 0s - loss: 7.8660 - acc: 0.4870
 epoch_end
13400/13400 [==============================] - 0s 2us/sample - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6666 - val_acc: 0.5000
Epoch 4/25
 1000/13400 [=>............................] - ETA: 0s - loss: 7.5900 - acc: 0.5050
 epoch_end
13400/13400 [==============================] - 0s 2us/sample - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6666 - val_acc: 0.5000
Epoch 

In [40]:
history_own.history

{'loss': [0.8613519075396123,
  0.7646200962242351,
  0.7632974074561316,
  0.7593458646499732,
  0.7615424006863205],
 'acc': [0.49589553, 0.5159702, 0.5210448, 0.5185075, 0.5156717],
 'val_loss': [0.7348190634539633,
  0.708384757834402,
  0.7773649608750235,
  0.6992647547568336,
  0.7293022869753115],
 'val_acc': [0.5386364, 0.5386364, 0.5386364, 0.5, 0.5]}

In [43]:
history_own = LossHistory()
optimizer = tf.keras.optimizers.SGD(0.01)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=1, validation_data=(X_test,Y_test), batch_size=1, callbacks=[history_own])
u=model.predict(X_train)
print(u)
#predict(x,batch_size=None,verbose=0,steps=None,callbacks=None,max_queue_size=10,workers=1,use_multiprocessing=False)

Train on 13400 samples, validate on 6600 samples
train_begin
13337/13400 [============================>.] - ETA: 0s - loss: 0.7579 - acc: 0.5219
 epoch_end
13400/13400 [==============================] - 13s 995us/sample - loss: 0.7577 - acc: 0.5225 - val_loss: 0.6999 - val_acc: 0.5386
[[3.9818615e-01 6.0081124e-01 1.2541220e-04 ... 1.2544045e-04
  1.2542572e-04 1.2543997e-04]
 [3.9818615e-01 6.0081124e-01 1.2541220e-04 ... 1.2544045e-04
  1.2542572e-04 1.2543997e-04]
 [3.9818615e-01 6.0081124e-01 1.2541220e-04 ... 1.2544045e-04
  1.2542572e-04 1.2543997e-04]
 ...
 [3.9818615e-01 6.0081124e-01 1.2541220e-04 ... 1.2544045e-04
  1.2542572e-04 1.2543997e-04]
 [7.2471070e-01 2.6788750e-01 9.2581153e-04 ... 9.2602440e-04
  9.2591927e-04 9.2600763e-04]
 [3.9818615e-01 6.0081124e-01 1.2541220e-04 ... 1.2544045e-04
  1.2542572e-04 1.2543997e-04]]


In [44]:
u[0]

array([3.9818615e-01, 6.0081124e-01, 1.2541220e-04, 1.2466905e-04,
       1.2541508e-04, 1.2544426e-04, 1.2540504e-04, 1.2544045e-04,
       1.2542572e-04, 1.2543997e-04], dtype=float32)

In [48]:
model.summary()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
dense_138 (Dense)            (None, 2)                 6         
_________________________________________________________________
dense_139 (Dense)            (None, 2)                 6         
_________________________________________________________________
dense_140 (Dense)            (None, 2)                 6         
_________________________________________________________________
dense_141 (Dense)            (None, 2)                 6         
_________________________________________________________________
dense_142 (Dense)            (None, 2)                 6         
_________________________________________________________________
dense_143 (Dense)            (None, 2)                 6  